In [1]:
# Cpython：内置类型的方法不会调用子类覆盖的方法
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
# 继承至dict的__init__方法，会忽略覆盖的__setitem__方法
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
# 继承至dict的update方法，也会忽略覆盖的__setitem__方法
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [5]:
# 菱形问题：由不相关的祖先类实现同名方法引发的命名冲突
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [7]:
d = D()
d.pong()

pong: <__main__.D object at 0x00000221CE770970>


In [8]:
C.pong(d)

PONG: <__main__.D object at 0x00000221CE770970>


In [9]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [10]:
d.ping()

ping: <__main__.D object at 0x00000221CE770970>
post-ping: <__main__.D object at 0x00000221CE770970>


In [11]:
d.pingpong()

ping: <__main__.D object at 0x00000221CE770970>
post-ping: <__main__.D object at 0x00000221CE770970>
ping: <__main__.D object at 0x00000221CE770970>
pong: <__main__.D object at 0x00000221CE770970>
pong: <__main__.D object at 0x00000221CE770970>
PONG: <__main__.D object at 0x00000221CE770970>


In [13]:
import tkinter
tkinter.Text.__mro__

(tkinter.Text,
 tkinter.Widget,
 tkinter.BaseWidget,
 tkinter.Misc,
 tkinter.Pack,
 tkinter.Place,
 tkinter.Grid,
 tkinter.XView,
 tkinter.YView,
 object)

In [14]:
# 处理多重继承
# 01. 把接口继承和实现继承区分开
# 继承接口，创建子类型，实现是什么关系
# 继承实现，通过重用避免代码重复
# 02. 使用抽象基类显式表示接口
# 如果类的作用是定义接口，应该明确把他定义为抽象基类
# 03. 通过混入（mixin）重用代码
# 如果一个类的作用是为多个不相关的子类提供方法实现
# 从而实现重用，但不体现‘是什么’关系，应该把这个类明确定义为混入类（mixin class）
# 04. 在名称中明确指明混入
# 05. 抽象基类可以作为混入，反之则不成立
# 06. 不要子类化多个具体的类
# 07. 为用户提供聚合类
# 08. 优先使用对象组合，而不是类继承